# AruCo marker video for multi-cam synchnonization

This is a simplified version of [GoPro Precision Date and Time (Local)](https://gopro.github.io/labs/control/precisiontime/).  Instead of displaying a QR code representing the current time in microseconds, the video in this example shows an AruCo marker at each frame.


In [1]:
%matplotlib notebook
import sys, os, cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

class AruCoMaker:
    def __init__(self, aruco_dict, dict_size, square_length):
        self.aruco_dict = aruco_dict
        self.dict_size = dict_size
        self.square_length = square_length
    
    def __getitem__(self, idx):
        idx %= self.dict_size
        return cv2.aruco.drawMarker(self.aruco_dict, idx, self.square_length)
    
class ChAruCoDiamondMaker:
    def __init__(self, aruco_dict, dict_size, square_length, marker_length):
        self.aruco_dict = aruco_dict
        self.dict_size = dict_size
        self.square_length = square_length
        self.marker_length = marker_length
    
    def __getitem__(self, idx):
        diamond_marker_ids = np.array([0, 1, 2, 3], dtype=int)
        diamond_marker_ids += idx
        diamond_marker_ids %= self.dict_size
        return cv2.aruco.drawCharucoDiamond(self.aruco_dict, diamond_marker_ids, self.square_length, self.marker_length)

def padto(image, width, height):
    ih, iw = image.shape[:2]
    buf = np.ones((height, width, 3), dtype=np.uint8) * 255
    dx = (width-iw)//2
    dy = (height-ih)//2
    buf[dy:dy+ih,dx:dx+iw,:] = image[:,:,None]
    return buf


In [2]:
from IPython.display import Video

duration = 60 * 5 # 5 min
video_width = 1920
video_height = 1080

aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
aruco_dict_size = 250

video_fps = [15, 30]
output_base = ["output-aruco", "output-diamond"]
maker = [AruCoMaker(aruco_dict, aruco_dict_size, 800),
         ChAruCoDiamondMaker(aruco_dict, aruco_dict_size, 300, 180)]

# opencv-python by pip does not suppert 'h264'.
codec = cv2.VideoWriter_fourcc(*'mp4v')


for fps in video_fps:
    for ba, ma in zip(output_base, maker):
        video = cv2.VideoWriter(ba + '.mp4', codec, fps, (video_width, video_height))
        for i in tqdm(range(duration*fps), desc=f'{ba}@{fps}Hz'):
            i = i % aruco_dict_size
            buf = padto(ma[i], video_width, video_height)
            cv2.putText(buf, f'DICT_6X6_250 #{i} @ {video_fps}Hz', (100, video_height-50), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 5, cv2.LINE_AA)
            video.write(buf)
        video.release()
        !ffmpeg -loglevel quiet -i "{ba}.mp4" -c:v libx265 -crf 22 -tag:v hvc1 -y "{ba}-{fps}hz.mp4"
        #Video('{ba}-{fps}hz.mp4', embed=True)

"""
for i in tqdm(range(duration*video_fps)):
    i = i % 250
    image = cv2.aruco.drawMarker(aruco_dict, i, square_length)
    buf = np.ones((video_height, video_width, 3), dtype=np.uint8) * 255
    dx = (video_width-square_length)//2
    dy = (video_height-square_length)//2
    buf[dy:dy+square_length,dx:dx+square_length,:] = image[:,:,None]
    
    cv2.putText(buf, f'DICT_6X6_250 #{i} @ {video_fps}Hz', (dx, video_height-50), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 5, cv2.LINE_AA)
    
    video.write(buf)

video.release()
"""


output-aruco@15Hz:   0%|          | 0/4500 [00:00<?, ?it/s]

x265 [info]: HEVC encoder version 3.5+1-f0c1022b6
x265 [info]: build info [Linux][GCC 8.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-4 (Main tier)
x265 [info]: Thread pool created using 20 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 4 / wpp(17 rows)
x265 [info]: Coding QT: max CU size, min CU size : 64 / 8
x265 [info]: Residual QT: max TU size, max depth : 32 / 1 inter / 1 intra
x265 [info]: ME / range / subpel / merge         : hex / 57 / 2 / 3
x265 [info]: Keyframe min / max / scenecut / bias  : 15 / 250 / 40 / 5.00 
x265 [info]: Lookahead / bframes / badapt        : 20 / 4 / 2
x265 [info]: b-pyramid / weightp / weightb       : 1 / 1 / 0
x265 [info]: References / ref-limit  cu / depth  : 3 / off / on
x265 [info]: AQ: mode / str / qg-size / cu-tree  : 2 / 1.0 / 32 / 1
x265 [info]: Rate Control / qCompress        

output-diamond@15Hz:   0%|          | 0/4500 [00:00<?, ?it/s]

x265 [info]: HEVC encoder version 3.5+1-f0c1022b6
x265 [info]: build info [Linux][GCC 8.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-4 (Main tier)
x265 [info]: Thread pool created using 20 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 4 / wpp(17 rows)
x265 [info]: Coding QT: max CU size, min CU size : 64 / 8
x265 [info]: Residual QT: max TU size, max depth : 32 / 1 inter / 1 intra
x265 [info]: ME / range / subpel / merge         : hex / 57 / 2 / 3
x265 [info]: Keyframe min / max / scenecut / bias  : 15 / 250 / 40 / 5.00 
x265 [info]: Lookahead / bframes / badapt        : 20 / 4 / 2
x265 [info]: b-pyramid / weightp / weightb       : 1 / 1 / 0
x265 [info]: References / ref-limit  cu / depth  : 3 / off / on
x265 [info]: AQ: mode / str / qg-size / cu-tree  : 2 / 1.0 / 32 / 1
x265 [info]: Rate Control / qCompress        

output-aruco@30Hz:   0%|          | 0/9000 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Re-encode

We can then use `ffmpeg` to encode MP4V to H264.  This makes the filesize smaller in general, and also allows embedding the video in the web browser.


In [5]:
from IPython.display import Video
!ffmpeg -loglevel quiet -i "{output_base}.mp4" -c:v libx265 -crf 22 -tag:v hvc1 -y "{output_base}-{video_fps}hz.mp4"
Video('{output_base}-{video_fps}hz.mp4', embed=True)

x265 [info]: HEVC encoder version 3.5+1-f0c1022b6
x265 [info]: build info [Linux][GCC 8.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-4 (Main tier)
x265 [info]: Thread pool created using 20 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 4 / wpp(17 rows)
x265 [info]: Coding QT: max CU size, min CU size : 64 / 8
x265 [info]: Residual QT: max TU size, max depth : 32 / 1 inter / 1 intra
x265 [info]: ME / range / subpel / merge         : hex / 57 / 2 / 3
x265 [info]: Keyframe min / max / scenecut / bias  : 15 / 250 / 40 / 5.00 
x265 [info]: Lookahead / bframes / badapt        : 20 / 4 / 2
x265 [info]: b-pyramid / weightp / weightb       : 1 / 1 / 0
x265 [info]: References / ref-limit  cu / depth  : 3 / off / on
x265 [info]: AQ: mode / str / qg-size / cu-tree  : 2 / 1.0 / 32 / 1
x265 [info]: Rate Control / qCompress        

# Diamond version

In [3]:
%matplotlib notebook
import sys, os, cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

duration = 60 * 5 # 5 min
video_fps = 15
output_base = "output-diamond"


video_width = 1920
video_height = 1080
square_length = 200
marker_length = 120
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

# opencv-python by pip does not suppert 'h264'.
codec = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_base + '.mp4', codec, video_fps, (video_width, video_height))

for i in tqdm(range(duration*video_fps)):
    diamond_marker_ids = np.array([0, 1, 2, 3], dtype=int)
    diamond_marker_ids += i
    diamond_marker_ids %= 250
    image = cv2.aruco.drawCharucoDiamond(aruco_dict, diamond_marker_ids, square_length, marker_length)
    ih, iw = image.shape[:2]
    buf = np.ones((video_height, video_width, 3), dtype=np.uint8) * 255
    dx = (video_width-iw)//2
    dy = (video_height-ih)//2
    buf[dy:dy+ih,dx:dx+iw,:] = image[:,:,None]
    
    cv2.putText(buf, f'DICT_6X6_250 {diamond_marker_ids} @ {video_fps}Hz', (dx, video_height-50), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 5, cv2.LINE_AA)
    
    video.write(buf)

video.release()


  0%|          | 0/4500 [00:00<?, ?it/s]

In [4]:
from IPython.display import Video
!ffmpeg -loglevel quiet -i "{output_base}.mp4" -c:v libx265 -crf 22 -tag:v hvc1 -y "{output_base}-{video_fps}hz.mp4"
Video('{output_base}-{video_fps}hz.mp4', embed=True)

## AruCo detection

Given two videos capturing a same AruCo video, this example first detects the marker from each frame, and tries to find the time offset between two videos.


In [67]:
INPUT_VIDEO_1 = "input1.mp4"
INPUT_VIDEO_2 = "input4.mp4"

# returns detected id or -1 at each frame
def get_frame_ids(video_file_name):
    video = cv2.VideoCapture(video_file_name)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    IDS = []
    CORNERS = []
    for i in tqdm(range(frame_count)):
        ret, frame = video.read()
        assert frame is not None

        corners, ids, rejectedCandidates = cv2.aruco.detectMarkers(frame, aruco_dict)
        if ids is not None and len(ids) == 1:
            IDS.append(ids[0,0])
        else:
            IDS.append(-1)

    return np.array(IDS)

def get_marker_segment(ids, break_count=5):
    begin = -1
    end = -1
    count = 0
    for i in range(len(ids)):
        if begin < 0 and ids[i] >= 0:
            begin = i
        elif ids[i] >= 0:
            count = 0
            continue
        else:
            count += 1
            if count >= break_count:
                end = i - count
                break
    return begin, end

id1 = get_frame_ids(INPUT_VIDEO_1)
id2 = get_frame_ids(INPUT_VIDEO_2)

# length of "with marker" segment
b1, e1 = get_marker_segment(id1)
b2, e2 = get_marker_segment(id2)

print(f'Input 1, using AruCo markers from {b1} to {e1}')
print(f'Input 2, using AruCo markers from {b2} to {e2}')

# brute-force search


  0%|          | 0/1446 [00:00<?, ?it/s]

  0%|          | 0/1408 [00:00<?, ?it/s]

Input 1, using AruCo markers from 0 to 209
Input 2, using AruCo markers from 0 to 211


In [69]:
def get_marker_segment(ids, break_count=5):
    begin = -1
    end = -1
    count = 0
    for i in range(len(ids)):
        if begin < 0 and ids[i] >= 0:
            begin = i
        elif ids[i] >= 0:
            count = 0
            continue
        else:
            count += 1
            if count >= break_count:
                end = i - count
                break
    return begin, end
        
# length of "with marker" segment
b1, e1 = get_marker_segment(id1)
b2, e2 = get_marker_segment(id2)

print(f'Input 1, using AruCo markers from {b1} to {e1}')
print(f'Input 2, using AruCo markers from {b2} to {e2}')

def count_matched_frames(seq1, seq2):
    if len(seq1) < len(seq2):
        return np.sum(np.equal(seq1, seq2[:len(seq1)])), len(seq1)
    else:
        return np.sum(np.equal(seq1[:len(seq2)], seq2)), len(seq2)

#def count_matched_frames(seq1, seq2):
#    if len(seq1) < len(seq2):
#        return np.sum(np.linalg.norm(seq1 - seq2[:len(seq1)])), len(seq1)
#    else:
#        return np.sum(np.linalg.norm(seq1[:len(seq2)] - seq2)), len(seq2)
    
def brute_force_search(seq1, seq2):
    max_match = 0 #len(seq1) + len(seq2)
    max_match_offset = 0
    for i in range(len(seq2)-1):
        c, l = count_matched_frames(seq1, seq2[i:])
        if max_match < c:
            max_match = c
            max_match_offset = i

    for i in range(len(sub1)-1):
        c, l = count_matched_frames(seq1[i:], seq2)
        if max_match < c:
            max_match = c
            max_match_offset = -i
    
    return max_match_offset, max_match

sub1 = id1[b1:e1].astype(float)
sub2 = id2[b2:e2].astype(float)
sub1[sub1 < 0] = np.nan
sub2[sub2 < 0] = np.nan

offset, count = brute_force_search(sub1, sub2)
print(offset, count)
for i, j in zip(id1[33:], id2):
    print(i, j, i-j)


Input 1, using AruCo markers from 0 to 209
Input 2, using AruCo markers from 0 to 211
1 13
-1 183 -184
-1 184 -185
-1 184 -185
176 185 -9
177 185 -8
178 186 -8
-1 186 -187
-1 188 -189
181 188 -7
181 189 -8
182 189 -7
183 190 -7
184 190 -6
-1 191 -192
186 192 -6
186 192 -6
187 -1 188
188 193 -5
189 194 -5
190 194 -4
190 195 -5
191 195 -4
192 196 -4
192 196 -4
194 -1 195
194 197 -3
195 198 -3
196 198 -2
196 199 -3
198 200 -2
198 200 -2
199 -1 200
200 201 -1
200 202 -2
202 202 0
202 203 -1
203 203 0
204 204 0
204 -1 205
205 205 0
206 206 0
206 206 0
208 -1 209
208 207 1
209 208 1
210 208 2
211 209 2
211 210 1
212 210 2
213 211 2
214 211 3
214 212 2
215 212 3
216 213 3
217 213 4
217 214 3
218 215 3
219 215 4
220 216 4
220 216 4
221 217 4
222 217 5
222 218 4
223 218 5
223 219 4
225 219 6
225 220 5
226 220 6
227 221 6
227 222 5
228 222 6
229 223 6
230 223 7
230 -1 231
231 224 7
232 225 7
232 225 7
233 -1 234
234 226 8
-1 227 -228
235 227 8
-1 228 -229
237 228 9
237 229 8
-1 229 -230
239 230 

11 246 -235
12 247 -235
-1 247 -248
13 248 -235
-1 248 -249
14 249 -235
15 249 -234
15 0 15
16 0 16
16 0 16
17 1 16
17 2 15
17 2 15
18 2 16
-1 3 -4
19 4 15
20 4 16
20 5 15
20 5 15
21 6 15
22 6 16
22 7 15
23 7 16
23 7 16
24 8 16
24 9 15
25 9 16
25 10 15
25 10 15
26 11 15
26 11 15
27 11 16
27 12 15
28 13 15
28 13 15
29 14 15
29 14 15
